In [1]:
import datetime
import time

import simpy
import pandas as pd
import shapely

import openclsim
import openclsim.model
import opentnsim
import networkx as nx

import logging
logging.basicConfig()

In [2]:
# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
graph = nx.Graph()
my_env.graph = graph

In [3]:
# create a Site object based on desired mixin classes
Site = type(
    "Site",
    (
        openclsim.core.Identifiable,
        openclsim.core.Log,
        openclsim.core.Locatable,
        openclsim.core.HasContainer,
        openclsim.core.HasResource,
    ),
    {},
)

# create a TransportProcessingResource object based on desired mixin classes
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        opentnsim.core.Movable,
        openclsim.core.Identifiable,
        openclsim.core.Log,
        openclsim.core.HasResource,
        openclsim.core.HasContainer,
        openclsim.core.Processor,
        # capture extra metadata to make sure we don't have leftover arguments
        opentnsim.core.ExtraMetadata
    ),
    {},
)

In [4]:
# prepare input data for from_site (note: in this example we only need the start location)
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)

# prepare input data for to_site
location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)
data_to_site = {
    "env": my_env,
    "name": "to_site",
    "geometry": location_to_site,
    "capacity": 100,
    "level": 100
}
# instantiate to_site 
to_site = Site(**data_to_site)


In [5]:
# create a location based graph (with geometry as nodes)
my_env.graph.add_node(location_from_site, **{"geometry": location_from_site})
my_env.graph.add_node(location_to_site, **{"geometry": location_to_site})
my_env.graph.add_edge(location_from_site, location_to_site)


In [6]:
# prepare input data for vessel_01
data_vessel01 = {
    "env": my_env,
    "name": "vessel01",
    "geometry": location_from_site, 
    "capacity": 5,
    "v": 10,
    "route": [location_from_site, location_to_site]
}
# instantiate vessel_01 
vessel01 = TransportProcessingResource(**data_vessel01)
assert not vessel01.metadata

In [7]:
# initialise registry
registry = {}

In [8]:
activity = openclsim.model.MoveActivity(
    env=my_env,
    name="Move activity",
    registry=registry,
    mover=vessel01,
    destination=to_site,
)

In [9]:
# initate the simpy processes defined in the 'move activity' and run simpy
openclsim.model.register_processes([activity])
my_env.run()

mover vessel01 destination to_site
getting location from to_site
moving to POINT (4.25222222 52.11428333)
moving over  (<POINT (4.181 52.187)>, <POINT (4.252 52.114)>)
mover vessel01 is now at POINT (4.25222222 52.11428333)
mover vessel01 is now at POINT (4.25222222 52.11428333)


In [10]:
display(openclsim.plot.get_log_dataframe(activity))

Activity                  Timestamp  \
0  7017847c-7178-4b59-bc7d-95ae3bf5ba85 1970-01-01 00:00:00.000000   
1  7017847c-7178-4b59-bc7d-95ae3bf5ba85 1970-01-01 00:15:42.824591   

  ActivityState  
0         START  
1          STOP